In [1]:
import pandas as pd
import numpy as np

import re

In [2]:
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 500)
pd.set_option('display.max_colwidth', -1)

## read, info, head

In [3]:
usecols=['id','title','genres','keywords','overview','popularity','production_companies','release_date','release_month',
         'tagline','vote_average','vote_count','cast_name','producer','director']

movie_df = pd.read_csv('./data/clean_movie_plus_credit.csv', encoding='raw_unicode_escape', usecols=usecols)

In [4]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 15 columns):
genres                  4775 non-null object
id                      4803 non-null int64
keywords                4390 non-null object
overview                4803 non-null object
popularity              4803 non-null float64
production_companies    4451 non-null object
release_date            4803 non-null object
tagline                 3959 non-null object
title                   4803 non-null object
vote_average            4803 non-null float64
vote_count              4803 non-null int64
release_month           4803 non-null int64
cast_name               4757 non-null object
producer                3743 non-null object
director                4562 non-null object
dtypes: float64(2), int64(3), object(10)
memory usage: 562.9+ KB


In [5]:
movie_df.head()

,genres,id,keywords,overview,popularity,production_companies,release_date,tagline,title,vote_average,vote_count,release_month,cast_name,producer,director
0,"Action,Adventure,Fantasy,Science Fiction",19995,"culture clash,future,space war,space colony,society,space travel,futuristic,romance,space,alien,tribe,alien planet,cgi,marine,soldier,battle,love affair,anti war,power relations,mind and soul,3d","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.",150.437577,"Ingenious Film Partners,Twentieth Century Fox Film Corporation,Dune Entertainment,Lightstorm Entertainment",2009-12-10,Enter the World of Pandora.,Avatar,7.2,11800,12,"Sam Worthington,Zoe Saldana,Sigourney Weaver,Stephen Lang,Michelle Rodriguez,Giovanni Ribisi,Joel David Moore,CCH Pounder,Wes Studi,Laz Alonso,Dileep Rao,Matt Gerald,Sean Anthony Moran,Jason Whyte,Scott Lawrence,Kelly Kilgour,James Patrick Pitt,Sean Patrick Murphy,Peter Dillon,Kevin Dorman,Kelson Henderson,David Van Horn,Jacob Tomuri,Jon Curry,Luke Hawker,Woody Schultz,Peter Mensah,Sonia Yee,Jahnel Curfman,Ilram Choi,Kyla Warren,Lisa Roumain,Debra Wilson,Chris Mala,Taylor Kibby,Jodie Landau,Julie Lamm,Cullen B. Madden,Joseph Brady Madden,Frankie Torres,Austin Wilson,Sara Wilson,Lucy Briant,Nathan Meister,Gerry Blair,Matthew Chamberlain,Paul Yates,Wray Wilson,James Gaylyn,Melvin Leno Clark III,Carvon Futrell,Brandon Jelkes,Micah Moch,Hanniyah Muhammad,Christopher Nolen,Christa Oliver,April Marie Thomas,Bravita A. Threatt,Colin Bleasdale,Mike Bodnar,Matt Clayton,Nicole Dionne,Jamie Harrison,Allan Henry,Anthony Ingruber,Ashley Jeffery,Dean Knowsley,Terry Notary,Kai Pantano,Logan Pithyou,Stuart Pollock,Raja,Gareth Ruck,Rhian Sheehan,T. J. Storm,Jodie Taylor,Richard Whiteside,Nikie Zambo,Julene Renee","James Cameron,Jon Landau",James Cameron
1,"Adventure,Fantasy,Action",285,"ocean,drug abuse,exotic island,east india trading company,love of one's life,traitor,shipwreck,strong woman,ship,alliance,calypso,afterlife,fighter,pirate,swashbuckler,aftercreditsstinger","Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.",139.082615,"Walt Disney Pictures,Jerry Bruckheimer Films,Second Mate Productions",2007-05-19,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,5,"Johnny Depp,Orlando Bloom,Keira Knightley,Bill Nighy,Geoffrey Rush,Jack Davenport,Kevin McNally,Tom Hollander,Naomie Harris,Jonathan Pryce,Keith Richards,Lee Arenberg,Mackenzie Crook,Greg Ellis,David Bailie,Martin Klebba,David Schofield,Lauren Maher,Vanessa Branch,Angus Barnett,Giles New,Reggie Lee,Dominic Scott Kay,Takayo Fischer,David Meunier,Andy Beckwith,Peter Donald Badalamenti II,Christopher S. Capp,Keith Richards,Ghassan Massoud","Jerry Bruckheimer,Eric McLeod,Chad Oman,Peter Kohn,Pat Sandston",Gore Verbinski
2,"Action,Adventure,Crime",206647,"spy,based on novel,secret agent,sequel,mi6,british secret service,united kingdom","A cryptic message from BondÃÂ¢ÃÂÃÂs past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit to reveal the terrible truth behind SPECTRE.",107.376788,"Columbia Pictures,Danjaq,B24",2015-10-26,A Plan No One Escapes,Spectre,6.3,4466,10,"Daniel Craig,Christoph Waltz,Ralph Fiennes,Monica Bellucci,Ben Whishaw,Naomie Harris,Dave Bautista,Andrew Scott,Rory Kinnear,Jesper Christensen,Alessandro Cremona,Stephanie Sigman,Tenoch Huerta,Adriana Paz,Domenico Fortunato,Marco Zingaro,Stefano Elfi DiClaudia,Ian Bonar,Tam Williams,Richard Banham,Pip Carter,Simon Lenagan,Alessandro Bressanello,Marc Zinga,Brigitte Millar,Adel Bencherif,Gediminas Adomaitis,Peppe Lanzetta,Francesco Arca,Matteo Taranto,Emilio Aniba,Benito Sagredo,Dai Tabuchi,Geor

## Get only 3 actor's name

In [6]:
#Fill all NaN with empty string
movie_df.fillna('', inplace=True)

In [7]:
movie_df['cast_name'] = movie_df['cast_name'].apply(lambda x: str(x.split(',')[:3])[1:-1] if len(x.split(',')) > 3 else str(x))

In [8]:
# movie_df['cast_name']

## Recommendation base on Weighted Rating

I use the TMDB Ratings to come up with our Top Movies Chart. I will use IMDB's weighted rating formula to construct my chart. Mathematically, it is represented as follows:

![](https://image.ibb.co/jYWZp9/wr.png)

v is the number of votes for the movie <br>
m is the minimum votes required to be listed in the chart<br>
R is the average rating of the movie<br>
C is the mean vote across the whole report<br>

In [9]:
C= movie_df['vote_average'].mean()

#minimum vote required, only 10% of the highest vote_count 
m= movie_df['vote_count'].quantile(0.9)

#get 10% of top vote_count movies
q_movies = movie_df.copy().loc[movie_df['vote_count'] >= m]
q_movies.shape

(481, 15)

In [10]:
#function for count weight rating
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [11]:
#insert weight rating into new columns ['score']
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

#sort movies base on weigth rating
q_movies = q_movies.sort_values('score', ascending=False)

In [12]:
#get top 10 weigthed rating
q_movies.head(10)[['id','title','genres','production_companies','popularity','vote_average','vote_count','cast_name','producer','director','release_date',]]

,id,title,genres,production_companies,popularity,vote_average,vote_count,cast_name,producer,director,release_date
1881,278,The Shawshank Redemption,"Drama,Crime",Castle Rock Entertainment,136.747729,8.5,8205,"'Tim Robbins', 'Morgan Freeman', 'Bob Gunton'",Niki Marvin,Frank Darabont,1994-09-23
662,550,Fight Club,Drama,"Regency Enterprises,Fox 2000 Pictures,Taurus Film,Linson Films,Atman Entertainment,Knickerbocker Films",146.757391,8.3,9413,"'Edward Norton', 'Brad Pitt', 'Meat Loaf'","Art Linson,Ross Grayson Bell",David Fincher,1999-10-15
65,155,The Dark Knight,"Drama,Action,Crime,Thriller","DC Comics,Legendary Pictures,Warner Bros.,DC Entertainment,Syncopy",187.322927,8.2,12002,"'Christian Bale', 'Heath Ledger', 'Aaron Eckhart'","Charles Roven,Christopher Nolan,Emma Thomas,Lorne Orleans",Christopher Nolan,2008-07-16
3232,680,Pulp Fiction,"Thriller,Crime","Miramax Films,A Band Apart,Jersey Films",121.463076,8.3,8428,"'John Travolta', 'Samuel L. Jackson', 'Uma Thurman'",Lawrence Bender,Quentin Tarantino,1994-10-08
96,27205,Inception,"Action,Thriller,Science Fiction,Mystery,Adventure","Legendary Pictures,Warner Bros.,Syncopy",167.583710,8.1,13752,"'Leonardo DiCaprio', 'Ellen Page', 'Tom Hardy'","Christopher Nolan,Emma Thomas",Christopher Nolan,2010-07-14
3337,238,The Godfather,"Drama,Crime","Paramount Pictures,Alfran Productions",143.659698,8.4,5893,"'Marlon Brando', 'Al Pacino', 'James Caan'",Albert S. Ruddy,Francis Ford Coppola,1972-03-14
95,157336,Interstellar,"Adventure,Drama,Science Fiction","Paramount Pictures,Legendary Pictures,Warner Bros.,Syncopy,Lynda Obst Productions",724.247784,8.1,10867,"'Matthew McConaughey', 'Jessica Chastain', 'Anne Hathaway'","Christopher Nolan,Emma Thomas,Lynda Obst",Christopher Nolan,2014-11-05
809,13,Forrest Gump,"Comedy,Drama,Romance",Paramount Pictures,138.133331,8.2,7927,"'Tom Hanks', 'Robin Wright', 'Gary Sinise'","Wendy Finerman,Steve Tisch,Steve Starkey",Robert Zemeckis,1994-07-06
329,122,The Lord of the Rings: The Return of the King,"Adventure,Fantasy,Action","WingNut Films,New Line Cinema",123.630332,8.1,8064,"'Elijah Wood', 'Ian McKellen', 'Viggo Mortensen'","Barrie M. Osborne,Peter Jackson",Peter Jackson,2003-12-01
1990,1891,The Empire Strikes Back,"Adventure,Action,Science Fiction","Lucasfilm,Twentieth Century Fox Film Corporation",78.517830,8.2,5879,"'Mark Hamill', 'Harrison Ford', 'Carrie Fisher'","Gary Kurtz,Howard G. Kazanjian",Irvin Kershner,1980-05-17


This model can give us Top 10 Movies in the TMDB list but we need a more flexible function so more film can be recommended.

## Recomendation base on Overview or Keywords

### base on Overview

In [13]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

def create_cosine_sim(df, col):
    #Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
    tfidf = TfidfVectorizer(stop_words='english')

    #Construct the required TF-IDF matrix by fitting and transforming the data
    tfidf_matrix = tfidf.fit_transform(df[col])
    
    # Compute the cosine similarity matrix
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    
    return cosine_sim

def get_recommendations(df, title, cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie (pair of the index and the similarity scores)
    # by taking 1 row that represent the movie in cosine_sim matrix
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the similarity scores, so we can get the most similar movie with the title that was given
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies, exclude index 0 because it is the title that was given
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['title'].iloc[movie_indices]

In [14]:
cosine_sim = create_cosine_sim(movie_df, 'overview')
cosine_sim.shape

(4803, 4803)

In [15]:
#Construct a reverse map of indices and movie titles, so we can get the index base on movie's title
indices = pd.Series(movie_df.index, index=movie_df['title']).drop_duplicates()

In [49]:
def create_df_overview(input_title):
    movie_title = get_recommendations(movie_df, input_title, cosine_sim)
    movie_rank = pd.Series([1,2,3,4,5,6,7,8,9,10], index=movie_title.index, name='Rank')
    recommendation_df = pd.DataFrame([movie_title,movie_rank]).transpose()
    movie_recommendation = pd.merge(movie_df[['title','overview']],recommendation_df, left_on='title', right_on = 'title')
    return movie_recommendation.sort_values('Rank')[['Rank','title','overview']]

In [51]:
create_df_overview('The Dark Knight Rises')

,Rank,title,overview
1,1,The Dark Knight,"Batman raises the stakes in his war on crime. With the help of Lt. Jim Gordon and District Attorney Harvey Dent, Batman sets out to dismantle the remaining criminal organizations that plague the streets. The partnership proves to be effective, but they soon find themselves prey to a reign of chaos unleashed by a rising criminal mastermind known to the terrified citizens of Gotham as the Joker."
4,2,Batman Forever,"The Dark Knight of Gotham City confronts a dastardly duo: Two-Face and the Riddler. Formerly District Attorney Harvey Dent, Two-Face believes Batman caused the courtroom accident which left him disfigured on one side. And Edward Nygma, computer-genius and former employee of millionaire Bruce Wayne, is out to get the philanthropist; as The Riddler. Former circus acrobat Dick Grayson, his family killed by Two-Face, becomes Wayne's ward and Batman's new partner Robin."
5,3,Batman Returns,"Having defeated the Joker, Batman now faces the Penguin - a warped and deformed individual who is intent on being accepted into Gotham society. Crooked businessman Max Schreck is coerced into helping him become Mayor of Gotham and they both attempt to expose Batman in a different light. Selina Kyle, Max's secretary, is thrown from the top of a building and is transformed into Catwoman - a mysterious figure who has the same personality disorder as Batman. Batman must attempt to clear his name, all the time deciding just what must be done with the Catwoman."
7,4,Batman,"The Dark Knight of Gotham City begins his war on crime with his first major enemy being the clownishly homicidal Joker, who has seized control of Gotham's underworld."
8,4,Batman,The Dynamic Duo faces four super-villains who plan to hold the world for ransom with the help of a secret invention that instantly dehydrates people.
10,5,"Batman: The Dark Knight Returns, Part 2",Batman has stopped the reign of terror that The Mutants had cast upon his city. Now an old foe wants a reunion and the government wants The Man of Steel to put a stop to Batman.
2,6,Batman Begins,"Driven by tragedy, billionaire Bruce Wayne dedicates his life to uncovering and defeating the corruption that plagues his home, Gotham City. Unable to work within the system, he instead creates a new identity, a symbol of fear for the criminal underworld - The Batman."
9,7,Slow Burn,"A district attorney (Ray Liotta) is involved in a 24-hour showdown with a gang leader (LL Cool J) and is, at the same time, being manipulated by an attractive assistant district attorney (Jolene Blalock) and a cryptic stranger."
6,8,JFK,New Orleans District Attorney Jim Garrison discovers there's more to the Kennedy assassination than the official story.
3,9,Batman & Robin,"Along with crime-fighting partner Robin and new recruit Batgirl, Batman battles the dual threat of frosty genius Mr. Freeze and homicidal horticulturalist Poison Ivy. Freeze plans to put Gotham City on ice, while Ivy tries to drive a wedge between the dynamic duo."


In [50]:
create_df_overview('The Lord of the Rings: The Return of the King')

,Rank,title,overview
4,1,The Lord of the Rings: The Two Towers,"Frodo and Sam are trekking to Mordor to destroy the One Ring of Power while Gimli, Legolas and Aragorn search for the orc-captured Merry and Pippin. All along, nefarious wizard Saruman awaits the Fellowship members at the Orthanc Tower in Isengard."
3,2,The Lord of the Rings: The Fellowship of the Ring,"Young hobbit Frodo Baggins, after inheriting a mysterious ring from his uncle Bilbo, must leave his home in order to keep it from falling into the hands of its evil creator. Along the way, a fellowship is formed to protect the ringbearer and make sure that the ring arrives at its final destination: Mt. Doom, the only place where it can be destroyed."
2,3,Transformers,"Young teenager, Sam Witwicky becomes involved in the ancient struggle between two extraterrestrial factions of transforming robots ÃÂ¢ÃÂÃÂ the heroic Autobots and the evil Decepticons. Sam holds the clue to unimaginable power and the Decepticons will stop at nothing to retrieve it."
0,4,The Hobbit: The Desolation of Smaug,"The Dwarves, Bilbo and Gandalf have successfully escaped the Misty Mountains, and Bilbo has gained the One Ring. They all continue their journey to get their gold back from the Dragon, Smaug."
8,5,Night Watch,"Among normal humans live the ""Others"" possessing various supernatural powers. They are divided up into the forces of light and the forces of the dark, who signed a truce several centuries ago to end a devastating battle. Ever since, the forces of light govern the day while the night belongs to their dark opponents. In modern day Moscow the dark Others actually roam the night as vampires while a ""Night Watch"" of light forces, among them Anton, the movie's protagonist, try to control them and limit their outrage"
7,6,Once Upon a Time in the West,This classic western masterpiece is an epic film about a widow whose land and life are in danger as the railroad is getting closer and closer to taking them over. A mysterious harmonica player joins forces with a desperado to protect the woman and her land.
9,7,Sparkler,"Melba is a Californian trailer-park girl who is told to look for three kings by a phone psychic, and when she meets three guys - Trent, Brad and Joel traveling to Las Vegas, she decides they are those kings and joins them on a trip. In Vegas she meets her old high school pal Dottie."
1,8,Prince of Persia: The Sands of Time,"A rogue prince reluctantly joins forces with a mysterious princess and together, they race against dark forces to safeguard an ancient dagger capable of releasing the Sands of Time ÃÂ¢ÃÂÃÂ gift from the gods that can reverse time and allow its possessor to rule the world."
5,9,I Am Sam,"Sam has the mental capacity of a 7-year-old. He has a daughter with a homeless woman who abandons them when they leave the hospital, leaving Sam to raise Lucy on his own. But as Lucy grows up, Sam's limitations start to become a problem and the authorities take her away. Sam shames high-priced lawyer Rita into taking his case pro bono and in turn teaches her the value of love and family."
6,10,Ouija,A group of friends must confront their most terrifying fears when they awaken the dark powers of an ancient spirit board.


In [52]:
create_df_overview('The Lord of the Rings: The Two Towers')

,Rank,title,overview
3,1,The Lord of the Rings: The Return of the King,"Aragorn is revealed as the heir to the ancient kings as he, Gandalf and the other members of the broken fellowship struggle to save Gondor from Sauron's forces. Meanwhile, Frodo and Sam bring the ring closer to the heart of Mordor, the dark lord's realm."
2,2,The Lord of the Rings: The Fellowship of the Ring,"Young hobbit Frodo Baggins, after inheriting a mysterious ring from his uncle Bilbo, must leave his home in order to keep it from falling into the hands of its evil creator. Along the way, a fellowship is formed to protect the ringbearer and make sure that the ring arrives at its final destination: Mt. Doom, the only place where it can be destroyed."
0,3,Tangled,"When the kingdom's most wanted-and most charming-bandit Flynn Rider hides out in a mysterious tower, he's taken hostage by Rapunzel, a beautiful and feisty tower-bound teen with 70 feet of magical, golden hair. Flynn's curious captor, who's looking for her ticket out of the tower where she's been locked away for years, strikes a deal with the handsome thief and the unlikely duo sets off on an action-packed escapade, complete with a super-cop horse, an over-protective chameleon and a gruff gang of pub thugs."
5,4,Underclassman,A young detective goes undercover at an elite private school to destroy an international stolen car ring.
6,5,I Am Sam,"Sam has the mental capacity of a 7-year-old. He has a daughter with a homeless woman who abandons them when they leave the hospital, leaving Sam to raise Lucy on his own. But as Lucy grows up, Sam's limitations start to become a problem and the authorities take her away. Sam shames high-priced lawyer Rita into taking his case pro bono and in turn teaches her the value of love and family."
7,6,Steamboy,"After receiving a package from his Grandfather, Rei, a young inventor living in England during the mid-19th century, has his life thrown into disarray. The package contains a ""Steam Ball"", a device of incredible power, and a letter asking him to guard it. Rei must evade capture from the nefarious ""O'Hara Foundation"" who will do anything to steal the device and use it for their own nefarious means."
4,7,Krull,A prince and a fellowship of companions set out to rescue his bride from a fortress of alien invaders who have arrived on their home planet.
1,8,Transformers,"Young teenager, Sam Witwicky becomes involved in the ancient struggle between two extraterrestrial factions of transforming robots ÃÂ¢ÃÂÃÂ the heroic Autobots and the evil Decepticons. Sam holds the clue to unimaginable power and the Decepticons will stop at nothing to retrieve it."
9,9,Small Apartments,"When a clumsy deadbeat accidentally kills his landlord, he must do everything in his power to hide the body, only to find that the distractions of lust, the death of his beloved brother, and a crew of misfit characters force him on a journey where a fortune awaits him."
8,10,Out of Inferno,"On the hottest day in 50 years, a serious fire incident happened to a busy commercial tower, a gaggle of fire fighters with an indestructible enthusiasm are going to save lives."


### base on Keyword

In [20]:
#stemmer is used for reduce morphological affixes
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

In [21]:
#function for make set of unique values of a columns
def make_set(df,col):
    set_keywords = set()
    for keyword in df[col].str.split(',').values:
        if isinstance(keyword, float): continue
        set_keywords = set_keywords.union(keyword)
    return set_keywords

#function for make a dictionary from a columns
def count_word(df, col):
    list_word = make_set(df,col)
    #make dict for count words
    keyword_count = dict()
    
    #fill dict key with words set default value to 0
    for i in list_word: keyword_count[i] = 0
    
    #loop all data from column, split all the word, set it as a list
    for list_key in df[col].str.split(','):  
        #ignore NaN
        if type(list_key) == float and pd.isnull(list_key): continue  
        #loop all tha data, remove word that's not in our list_word
        for key in [s for s in list_key if s in list_word]:
            #add 1 count for every word that found
            keyword_count[key] += 1
            
    #return unsorted dict
    return keyword_count

In [22]:
#check keyword that only appears once, twice, three times, and four times
sum1 = sum2 = sum3 = sum4 = 0
for i in count_word(movie_df,'keywords').values():
    if i == 1 : sum1 += 1
    elif i == 2 : sum2 += 1
    elif i == 3 : sum3 += 1
    else: sum4 += 1
print(sum1 , sum2 , sum3, sum4)

5273 1547 804 2145


In [23]:
#since there are 5273 keywords that appears once and 1547 that appears twice
#to reduce the size of matrix, we will only use key words that appears three times or more
keywords_list = []
for k,v in zip(count_word(movie_df,'keywords').keys(),count_word(movie_df,'keywords').values()):
    if v > 2: keywords_list.append(k)

In [24]:
#function to reduce keywords and remove space for keywords that have 2 or more words and replace , with space
def filter_keywords(param):
    words = []
    for i in param.split(','):
#         print(i)
        if i in keywords_list:
            stem_i = stemmer.stem(i)
            words.append(stem_i)
    return re.sub(r' |\'|\[|\]','',str(words)).lower().replace(',',' ')

In [25]:
movie_df['keywords'] = movie_df['keywords'].apply(filter_keywords)

In [26]:
cosine_sim1 = create_cosine_sim(movie_df, 'keywords')

In [27]:
def create_df_keywords(input_movie):
    movie_title = get_recommendations(movie_df, input_movie, cosine_sim1)
    movie_rank = pd.Series([1,2,3,4,5,6,7,8,9,10], index=movie_title.index, name='Rank')
    recommendation_df = pd.DataFrame([movie_title,movie_rank]).transpose()
    movie_recommendation = pd.merge(movie_df[['title','keywords']],recommendation_df, left_on='title', right_on = 'title')
    return movie_recommendation.sort_values('Rank')[['Rank','title','keywords']]

In [28]:
create_df_keywords('The Dark Knight Rises')

,Rank,title,keywords
1,1,The Dark Knight,dccomic crimefight secretident scarecrow sadism chao gothamc vigilant superhero basedoncomicbook tragichero organizedcrim criminalmastermind districtattorney imax supervillain superpow batman
2,2,Batman Begins,himalaya martialart dccomic crimefight secretident undercov hero lossoffath societi gothamc vigilant superhero basedoncomicbook rivalri tragichero ninja goodvsevil crime superpow escapad mastervillain loveandrom
3,3,Batman & Robin,doublelif dccomic dualident crimefight fictionalplac gothamc superhero superpow
5,4,Batman,doublelif dccomic dualident crimefight fictionalplac gothamc superhero superpow
6,4,Batman,submarin dccomic missil sharkattack rescu shark superpow
4,5,Batman Returns,holiday corrupt doublelif dccomic crimefight hallucin christmastre gothamc vigilant superhero violenc darkhero supervillain superpow
7,6,Kick-Ass 2,crimefight secretident marvelcom superhero aftercreditssting teensuperhero
0,7,Batman v Superman: Dawn of Justice,dccomic vigilant superhero basedoncomicbook reveng superpow dcextendedunivers
8,8,Arbitrage,cover-up
10,9,Defendor,crimefight delus superhero


In [29]:
create_df_keywords('The Lord of the Rings: The Return of the King')

,Rank,title,keywords
0,1,The Hobbit: The Battle of the Five Armies,corrupt elv dwarv orc middle-earth(tolkien) hobbit dragon battl unlikelyfriendship swordandsorceri
6,2,The Lord of the Rings: The Two Towers,elv orc middle-earth(tolkien) hobbit basedonnovel explos cave fort armi mission attack wizard ring swordandsorceri
5,3,The Lord of the Rings: The Fellowship of the Ring,elv dwarv orc middle-earth(tolkien) hobbit basedonnovel mountain firework castl volcano deathofafriend uncl mirror wizard swordandsorceri
2,4,The Hobbit: An Unexpected Journey,riddl elv dwarv orc middle-earth(tolkien) hobbit mountain wizard journey ring goblin courag giant tunnel invis
1,5,The Hobbit: The Desolation of Smaug,elv dwarv orc hobbit dragon wizard swordandsorceri
3,6,Warcraft,videogam elv orc magic chase basedoncomicbook sorcer fictionalwar basedonvideogam wizard swordandsorceri
9,7,Good Kill,pilot suspicion
8,8,The Four Feathers,islam sex loyalti braveri armi reveng honor murder escap soldier battl church violenc britain slave coloni militari
4,9,The Chronicles of Narnia: The Voyage of the Dawn Treader,basedonnovel magic goodvsevil king narnia fantasyworld quest
7,10,Winter's Tale,basedonnovel


In [30]:
create_df_keywords('Fight Club')

,Rank,title,keywords
2,1,The One,dualident
3,2,Se7en,self-fulfillingpropheci detect drugdeal rageandh insomnia investig polic serialkil
8,3,The Machinist,factori insomnia
1,4,After Earth,dystopia
4,5,Megiddo: The Omega Code 2,dystopia
7,6,Repo! The Genetic Opera,dystopia
9,7,Dead Man's Shoes,rageandh brother reveng punish
6,8,The Fault in Our Stars,amsterdam basedonnovel supportgroup cancer teenag starcrossedlov teendrama basedonyoungadultnovel
0,9,Wreck-It Ralph,supportgroup productplac bulli medal selfesteem curios aftercreditssting duringcreditssting
5,10,The New Guy,prison dualident ident loser highschool losangel duringcreditssting


This model can give us similar film base on overview or keywords, but it have a tendency to give us film with simillar title, since it only take overview or keywords into calculation.

## Recommendation Keywords, Genres, Director, Cast using CountVectorizer or TF-IDF

### CountVectorizer

In [31]:
movie_df['director'] = movie_df['director'].apply(lambda x: x.replace(',',' ').lower())
movie_df['director'] = movie_df['director'].apply(lambda x: x.lower().replace(' ',''))
movie_df['cast_name'] = movie_df['cast_name'].apply(lambda x: re.sub(r' |\'|\[|\]','',x.lower()).replace(',',' '))
movie_df['genres'] = movie_df['genres'].apply(lambda x: x.replace(',',' ').lower())

In [32]:
movie_df['soup'] = movie_df[['keywords','cast_name','director','genres']].apply(lambda x: ''.join(x['keywords']) + ' '+ ''.join(x['cast_name'])+ ' ' + ''.join(x['director']) +' '+ ''.join(x['genres']), axis=1)

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(movie_df['soup'])

In [34]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [35]:
def create_df_soup_cv(input_title):
    movie_title = get_recommendations(movie_df, input_title, cosine_sim2)
    movie_rank = pd.Series([1,2,3,4,5,6,7,8,9,10], index=movie_title.index, name='Rank')
    recommendation_df = pd.DataFrame([movie_title,movie_rank]).transpose()
    movie_recommendation = pd.merge(movie_df[['title','soup']],recommendation_df, left_on='title', right_on = 'title')
    return movie_recommendation.sort_values('Rank')[['Rank','title','soup']]

In [36]:
create_df_soup_cv('The Dark Knight Rises')

,Rank,title,soup
0,1,The Dark Knight,dccomic crimefight secretident scarecrow sadism chao gothamc vigilant superhero basedoncomicbook tragichero organizedcrim criminalmastermind districtattorney imax supervillain superpow batman christianbale heathledger aaroneckhart christophernolan drama action crime thriller
1,2,Batman Begins,himalaya martialart dccomic crimefight secretident undercov hero lossoffath societi gothamc vigilant superhero basedoncomicbook rivalri tragichero ninja goodvsevil crime superpow escapad mastervillain loveandrom christianbale michaelcaine liamneeson christophernolan action crime drama
8,3,Harsh Times,christianbale evalongoria chakaforman davidayer crime drama thriller action
9,4,Amidst the Devil's Wings,drama action crime
4,5,Takers,heist chrisbrown haydenchristensen mattdillon johnluessenhop action crime drama thriller
2,6,Kiss of Death,hoodlum davidcaruso samuell.jackson nicolascage barbetschroeder action crime drama thriller
5,7,Street Kings,polic losangel keanureeves forestwhitaker chrisevans davidayer action crime drama thriller
7,8,Romeo Is Bleeding,policeoper sexaddict polic mafiaboss suspens garyoldman lenaolin annabellasciorra petermedak action crime drama thriller
6,9,Trash,brazilian drama thriller ricksontevez eduardoluis gabrielweinstein stephendaldry adventure crime drama thriller
3,10,Stolen,taxidriv thief fbiag nicolascage joshlucas samigayle simonwest action crime drama thriller


In [37]:
create_df_soup_cv('The Lord of the Rings: The Return of the King')

,Rank,title,soup
6,1,The Lord of the Rings: The Two Towers,elv orc middle-earth(tolkien) hobbit basedonnovel explos cave fort armi mission attack wizard ring swordandsorceri elijahwood ianmckellen viggomortensen peterjackson adventure fantasy action
5,2,The Lord of the Rings: The Fellowship of the Ring,elv dwarv orc middle-earth(tolkien) hobbit basedonnovel mountain firework castl volcano deathofafriend uncl mirror wizard swordandsorceri elijahwood ianmckellen cateblanchett peterjackson adventure fantasy action
0,3,The Hobbit: The Battle of the Five Armies,corrupt elv dwarv orc middle-earth(tolkien) hobbit dragon battl unlikelyfriendship swordandsorceri martinfreeman ianmckellen richardarmitage peterjackson action adventure fantasy
2,4,The Hobbit: An Unexpected Journey,riddl elv dwarv orc middle-earth(tolkien) hobbit mountain wizard journey ring goblin courag giant tunnel invis ianmckellen martinfreeman richardarmitage peterjackson adventure fantasy action
1,5,The Hobbit: The Desolation of Smaug,elv dwarv orc hobbit dragon wizard swordandsorceri martinfreeman ianmckellen richardarmitage peterjackson adventure fantasy
3,6,Warcraft,videogam elv orc magic chase basedoncomicbook sorcer fictionalwar basedonvideogam wizard swordandsorceri paulapatton travisfimmel benfoster duncanjones action adventure fantasy
7,7,Conan the Barbarian,gladiat repay barbarian swordandsorceri arnoldschwarzenegger jamesearljones maxvonsydow johnmilius adventure fantasy action
4,8,The 13th Warrior,witch cave vike iraq war mission antoniobanderas vladimirkulich danielsouthern johnmctiernan adventure fantasy action
8,9,Red Sonja,monster swordplay queen marvelcom basedoncomicbook swordandsorceri arnoldschwarzenegger brigittenielsen sandahlbergman richardfleischer adventure fantasy action
9,10,Beastmaster 2: Through the Portal of Time,basedonnovel timetravel sequel psychotron warrior swordandsorceri marcsinger kariwuhrer sarahdouglas sylviotabet action adventure fantasy science fiction


In [38]:
create_df_soup_cv('The Hobbit: An Unexpected Journey')

,Rank,title,soup
0,1,The Hobbit: The Battle of the Five Armies,corrupt elv dwarv orc middle-earth(tolkien) hobbit dragon battl unlikelyfriendship swordandsorceri martinfreeman ianmckellen richardarmitage peterjackson action adventure fantasy
5,2,The Lord of the Rings: The Fellowship of the Ring,elv dwarv orc middle-earth(tolkien) hobbit basedonnovel mountain firework castl volcano deathofafriend uncl mirror wizard swordandsorceri elijahwood ianmckellen cateblanchett peterjackson adventure fantasy action
1,3,The Hobbit: The Desolation of Smaug,elv dwarv orc hobbit dragon wizard swordandsorceri martinfreeman ianmckellen richardarmitage peterjackson adventure fantasy
7,4,The Lord of the Rings: The Two Towers,elv orc middle-earth(tolkien) hobbit basedonnovel explos cave fort armi mission attack wizard ring swordandsorceri elijahwood ianmckellen viggomortensen peterjackson adventure fantasy action
6,5,The Lord of the Rings: The Return of the King,elv orc middle-earth(tolkien) basedonnovel suspicion braveri war honor troll brutal violenc ghost quest swordandsorceri elijahwood ianmckellen viggomortensen peterjackson adventure fantasy action
3,6,Warcraft,videogam elv orc magic chase basedoncomicbook sorcer fictionalwar basedonvideogam wizard swordandsorceri paulapatton travisfimmel benfoster duncanjones action adventure fantasy
9,7,The Borrowers,"dwarv lawyer littlepeopl ""childspointofview"" realest johngoodman jimbroadbent celiaimrie peterhewitt adventure fantasy action comedy family"
8,8,The Monkey King 2,aaronkwok gongli williamfeng soicheang action adventure fantasy
2,9,Prince of Persia: The Sands of Time,persia sandstorm basedonvideogam jakegyllenhaal gemmaarterton benkingsley mikenewell adventure fantasy action romance
4,10,Night at the Museum: Battle of the Smithsonian,museum duringcreditssting benstiller amyadams owenwilson shawnlevy adventure fantasy action comedy family


### TF-IDF

In [39]:
tfidf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = tfidf.fit_transform(movie_df['soup'])
cosine_sim3 = cosine_similarity(count_matrix, count_matrix)

In [40]:
def create_df_soup_td(input_title):
    movie_title = get_recommendations(movie_df, input_title, cosine_sim3)
    movie_rank = pd.Series([1,2,3,4,5,6,7,8,9,10], index=movie_title.index, name='Rank')
    recommendation_df = pd.DataFrame([movie_title,movie_rank]).transpose()
    movie_recommendation = pd.merge(movie_df[['title','soup']],recommendation_df, left_on='title', right_on = 'title')
    return movie_recommendation.sort_values('Rank')[['Rank','title','soup']]

In [41]:
create_df_soup_td('The Dark Knight Rises')

,Rank,title,soup
0,1,The Dark Knight,dccomic crimefight secretident scarecrow sadism chao gothamc vigilant superhero basedoncomicbook tragichero organizedcrim criminalmastermind districtattorney imax supervillain superpow batman christianbale heathledger aaroneckhart christophernolan drama action crime thriller
1,2,Batman Begins,himalaya martialart dccomic crimefight secretident undercov hero lossoffath societi gothamc vigilant superhero basedoncomicbook rivalri tragichero ninja goodvsevil crime superpow escapad mastervillain loveandrom christianbale michaelcaine liamneeson christophernolan action crime drama
3,3,Batman Returns,holiday corrupt doublelif dccomic crimefight hallucin christmastre gothamc vigilant superhero violenc darkhero supervillain superpow michaelkeaton dannydevito michellepfeiffer timburton action fantasy
2,4,Batman & Robin,doublelif dccomic dualident crimefight fictionalplac gothamc superhero superpow georgeclooney arnoldschwarzenegger umathurman joelschumacher action crime fantasy
6,5,The Prestige,competit secret obsess magic dyinganddeath classsocieti illus hostil class rivalri hughjackman christianbale michaelcaine christophernolan drama mystery thriller
10,6,Amidst the Devil's Wings,drama action crime
7,7,Batman,doublelif dccomic dualident crimefight fictionalplac gothamc superhero superpow jacknicholson michaelkeaton kimbasinger timburton fantasy action
8,7,Batman,submarin dccomic missil sharkattack rescu shark superpow adamwest burtward leemeriwether leslieh.martinson family adventure comedy science fiction crime
4,8,Hellboy,blackmag fistfight cover-up superhero paranormalphenomena demon occult duringcreditssting ronperlman selmablair rupertevans guillermodeltoro fantasy action science fiction
9,9,Harsh Times,christianbale evalongoria chakaforman davidayer crime drama thriller action


In [42]:
create_df_soup_td('The Dark Knight')

,Rank,title,soup
2,1,Batman Begins,himalaya martialart dccomic crimefight secretident undercov hero lossoffath societi gothamc vigilant superhero basedoncomicbook rivalri tragichero ninja goodvsevil crime superpow escapad mastervillain loveandrom christianbale michaelcaine liamneeson christophernolan action crime drama
0,2,The Dark Knight Rises,dccomic crimefight terrorist secretident burglar hostagedrama timebomb gothamc vigilant cover-up superhero villai tragichero terror destruct imax flood batman christianbale michaelcaine garyoldman christophernolan action crime drama thriller
5,3,Batman Returns,holiday corrupt doublelif dccomic crimefight hallucin christmastre gothamc vigilant superhero violenc darkhero supervillain superpow michaelkeaton dannydevito michellepfeiffer timburton action fantasy
10,4,Amidst the Devil's Wings,drama action crime
1,5,Batman v Superman: Dawn of Justice,dccomic vigilant superhero basedoncomicbook reveng superpow dcextendedunivers benaffleck henrycavill galgadot zacksnyder action adventure fantasy
3,6,Batman & Robin,doublelif dccomic dualident crimefight fictionalplac gothamc superhero superpow georgeclooney arnoldschwarzenegger umathurman joelschumacher action crime fantasy
6,7,Superman,savingtheworld journalist dccomic crimefight nuclearmissil galaxi superhero basedoncomicbook crimin sabotag northpol superpow superhumanstrength aftercreditssting christopherreeve marlonbrando margotkidder richarddonner action adventure fantasy science fiction
7,8,Batman,doublelif dccomic dualident crimefight fictionalplac gothamc superhero superpow jacknicholson michaelkeaton kimbasinger timburton fantasy action
8,8,Batman,submarin dccomic missil sharkattack rescu shark superpow adamwest burtward leemeriwether leslieh.martinson family adventure comedy science fiction crime
4,9,Batman Forever,riddl dccomic gothamc partner superhero brokenneck psychologist violenc crimin districtattorney millionair tiedup superpow disfigur fatherfigur valkilmer tommyleejones jimcarrey joelschumacher action crime fantasy


In [43]:
create_df_soup_td('The Lord of the Rings: The Return of the King')

,Rank,title,soup
5,1,The Lord of the Rings: The Two Towers,elv orc middle-earth(tolkien) hobbit basedonnovel explos cave fort armi mission attack wizard ring swordandsorceri elijahwood ianmckellen viggomortensen peterjackson adventure fantasy action
4,2,The Lord of the Rings: The Fellowship of the Ring,elv dwarv orc middle-earth(tolkien) hobbit basedonnovel mountain firework castl volcano deathofafriend uncl mirror wizard swordandsorceri elijahwood ianmckellen cateblanchett peterjackson adventure fantasy action
0,3,The Hobbit: The Battle of the Five Armies,corrupt elv dwarv orc middle-earth(tolkien) hobbit dragon battl unlikelyfriendship swordandsorceri martinfreeman ianmckellen richardarmitage peterjackson action adventure fantasy
2,4,The Hobbit: An Unexpected Journey,riddl elv dwarv orc middle-earth(tolkien) hobbit mountain wizard journey ring goblin courag giant tunnel invis ianmckellen martinfreeman richardarmitage peterjackson adventure fantasy action
1,5,The Hobbit: The Desolation of Smaug,elv dwarv orc hobbit dragon wizard swordandsorceri martinfreeman ianmckellen richardarmitage peterjackson adventure fantasy
3,6,Warcraft,videogam elv orc magic chase basedoncomicbook sorcer fictionalwar basedonvideogam wizard swordandsorceri paulapatton travisfimmel benfoster duncanjones action adventure fantasy
6,7,Conan the Barbarian,gladiat repay barbarian swordandsorceri arnoldschwarzenegger jamesearljones maxvonsydow johnmilius adventure fantasy action
9,8,Red Sonja,monster swordplay queen marvelcom basedoncomicbook swordandsorceri arnoldschwarzenegger brigittenielsen sandahlbergman richardfleischer adventure fantasy action
7,9,Con Air,prison ambush helicopt airport gasstat undercov mexicanstandoff braveri hijack escap shootout u.s.marsh lasvega explos brutal violenc convict desert deaag nicolascage johncusack johnmalkovich simonwest action thriller crime
8,10,The Last Witch Hunter,newyork witch upris witchhunt vindiesel roseleslie elijahwood breckeisner fantasy action adventure


This model still give us movie with similar title even thought we have combine cast, genre, keywords and director

## Recommendation Keywords, Genres, Director, Cast TD with Weight

In [44]:
#create cosine for every features
cosine_sim_genre = create_cosine_sim(movie_df, 'genres')
cosine_sim_cast = create_cosine_sim(movie_df, 'cast_name')
cosine_sim_keywords = create_cosine_sim(movie_df, 'keywords')
cosine_sim_director = create_cosine_sim(movie_df, 'director')

#combine the cosine, put more weight on cast and director, reduce genre's and keyword's weight
cosine_sim4 = ((cosine_sim_genre*0.2)+(cosine_sim_cast*0.35)+(cosine_sim_keywords*0.2)+(cosine_sim_director*0.25))

In [45]:
def create_df_td_weight(input_title):
    movie_title = get_recommendations(movie_df, input_title, cosine_sim4)
    movie_rank = pd.Series([1,2,3,4,5,6,7,8,9,10], index=movie_title.index, name='Rank')
    recommendation_df = pd.DataFrame([movie_title,movie_rank]).transpose()
    movie_recommendation = pd.merge(movie_df[['title','genres','cast_name','keywords','director']],recommendation_df, left_on='title', right_on = 'title')
    return movie_recommendation.sort_values('Rank')[['Rank','title','genres','cast_name','keywords','director']]

In [46]:
create_df_td_weight('The Dark Knight Rises')

,Rank,title,genres,cast_name,keywords,director
2,1,Batman Begins,action crime drama,christianbale michaelcaine liamneeson,himalaya martialart dccomic crimefight secretident undercov hero lossoffath societi gothamc vigilant superhero basedoncomicbook rivalri tragichero ninja goodvsevil crime superpow escapad mastervillain loveandrom,christophernolan
0,2,The Dark Knight,drama action crime thriller,christianbale heathledger aaroneckhart,dccomic crimefight secretident scarecrow sadism chao gothamc vigilant superhero basedoncomicbook tragichero organizedcrim criminalmastermind districtattorney imax supervillain superpow batman,christophernolan
4,3,The Prestige,drama mystery thriller,hughjackman christianbale michaelcaine,competit secret obsess magic dyinganddeath classsocieti illus hostil class rivalri,christophernolan
3,4,Insomnia,crime mystery thriller,alpacino robinwilliams hilaryswank,detect confess fbi homicid blackmail suspect love murder losangel teenag neo-noir,christophernolan
1,5,Inception,action thriller science fiction mystery adventure,leonardodicaprio ellenpage tomhardy,lossoflov dream kidnap sleep subconsci heist redempt,christophernolan
7,6,Harry Brown,thriller crime drama action,michaelcaine emilymortimer iainglen,self-defens widow,danielbarber
8,7,Memento,mystery thriller,guypearce joepantoliano markboonejunior,individu tattoo waitress amnesia motel insurancesalesman reveng memoryloss polaroid flashback neo-noir,christophernolan
6,8,Romeo Is Bleeding,action crime drama thriller,garyoldman lenaolin annabellasciorra,policeoper sexaddict polic mafiaboss suspens,petermedak
9,9,Harsh Times,crime drama thriller action,christianbale evalongoria chakaforman,,davidayer
5,10,Out of the Furnace,thriller drama crime,christianbale zoesaldana woodyharrelson,prison brotherbrotherrelationship drugdeal brother reveng drugus caraccid justic famili neo-noir,scottcooper


In [47]:
create_df_td_weight('The Lord of the Rings: The Return of the King')

,Rank,title,genres,cast_name,keywords,director
7,1,The Lord of the Rings: The Two Towers,adventure fantasy action,elijahwood ianmckellen viggomortensen,elv orc middle-earth(tolkien) hobbit basedonnovel explos cave fort armi mission attack wizard ring swordandsorceri,peterjackson
5,2,The Lord of the Rings: The Fellowship of the Ring,adventure fantasy action,elijahwood ianmckellen cateblanchett,elv dwarv orc middle-earth(tolkien) hobbit basedonnovel mountain firework castl volcano deathofafriend uncl mirror wizard swordandsorceri,peterjackson
0,3,The Hobbit: The Battle of the Five Armies,action adventure fantasy,martinfreeman ianmckellen richardarmitage,corrupt elv dwarv orc middle-earth(tolkien) hobbit dragon battl unlikelyfriendship swordandsorceri,peterjackson
3,4,The Hobbit: An Unexpected Journey,adventure fantasy action,ianmckellen martinfreeman richardarmitage,riddl elv dwarv orc middle-earth(tolkien) hobbit mountain wizard journey ring goblin courag giant tunnel invis,peterjackson
1,5,The Hobbit: The Desolation of Smaug,adventure fantasy,martinfreeman ianmckellen richardarmitage,elv dwarv orc hobbit dragon wizard swordandsorceri,peterjackson
2,6,King Kong,adventure drama action,naomiwatts jackblack adrienbrody,filmbusi screenplay showbusi filmmak filmproduc exoticisland monster indigen ship dinosaur,peterjackson
6,7,The Lovely Bones,fantasy drama,rachelweisz markwahlberg susansarandon,rape 1970s evid tree afterlif lossofdaught serialkil corps pedophil teenagelov griev basedonyoungadultnovel,peterjackson
9,8,Heavenly Creatures,drama fantasy,melanielynskey katewinslet sarahpeirse,mother sex secret obsess literatur nuditi fantasi passion love friend murder independentfilm lesbian true relationship,peterjackson
8,9,The Last Witch Hunter,fantasy action adventure,vindiesel roseleslie elijahwood,newyork witch upris witchhunt,breckeisner
4,10,Warcraft,action adventure fantasy,paulapatton travisfimmel benfoster,videogam elv orc magic chase basedoncomicbook sorcer fictionalwar basedonvideogam wizard swordandsorceri,duncanjones


Finally, we can get a more variation, but since we put more weight on cast an director, movies that appear tend to come from similar director. If, the movie have sequels from the same director, it will guarantee that the movie's sequel will come up.